In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
counties_shp = gpd.read_file("data/1_raw/counties_2020/tl_2020_us_county.shp")
counties_csv = pd.read_csv("data/1_raw/counties.csv")

In [15]:
def join_data(shp, csv, left_on, right_on): 
    shp = shp.astype({key: 'int32' for key in left_on})
    merged = pd.merge(shp, csv, left_on=left_on, right_on=right_on, how="outer").fillna(0)
    return merged

def get_cols(merged, cols=["H1_001N", "H1_003N", "geometry"]):
    merged = merged[cols]
#     merged = merged.rename(columns={"H1_001N": "total_units", "H1_003N": "vacant_units"})
    merged["percent_vacant"] = 100.0 * merged["vacant_units"] / merged["total_units"]
    return merged

In [13]:
joined = get_cols(join_data(counties_shp, counties_csv, ["STATEFP","COUNTYFP"], ["state","county"]))
joined.to_file("data/2_joined/counties/counties.shp")

In [14]:
def read_data(f):
    shp = gpd.read_file("data/1_raw/" + f + "_2020/" + f + "_2020.shp")
    csv = pd.read_csv("data/1_raw/" + f + ".csv")
    
    return shp, csv

In [16]:
tracts_shp, tracts_csv = read_data("tracts")
tracts_joined = join_data(tracts_shp, tracts_csv, ["STATEFP", "COUNTYFP", "TRACTCE"], ["state", "county", "tract"])
get_cols(tracts_joined).to_file("data/2_joined/tracts/tracts.shp")

In [17]:
bgs_shp, bgs_csv = read_data("bgs")
bgs_joined = join_data(bgs_shp, bgs_csv, ["STATEFP", "COUNTYFP", "TRACTCE", "BLKGRPCE"], ["state", "county", "tract", "block group"])
get_cols(bgs_joined, cols=["total_units", "vacant_units", "geometry"]).to_file("data/2_joined/bgs/bgs.shp")

